# Sistema de Compressão

$ \frac{d\dot{m}}{dt} = \frac{A_1}{L_C}(\phi (N(t), \dot{m})P_1 - P_P(t)) $

$ \frac{d P_P}{dt} = \frac{C_1^2}{\nu _P}(\dot{m}(t) - \alpha (t) K_\nu \sqrt{P_P - P_{out}}) $

$ \begin{matrix} A_1 & = & 2.6\centerdot 10^{-3} m² \\
\nu _P & = & 2.0 m³ \\
L_C & = & 2.0 m \\
K_\nu & = & \frac{0.38 kg}{(kPa)^{0.5}s} \\
P_1 & = & 4.5 MPa \\
P{out} & = & 5.0 MPa \end{matrix}
$

$ \frac{d\dot{m}}{dt} = \frac{2.6\centerdot 10^{-3}}{2.0}(1.5\centerdot 4.5 - P_P) $

$ \frac{d P_P}{dt} = \frac{479.029^2}{2.0}(\dot{m} - \alpha {0.38} \sqrt{P_P - 5.0}) $

#### Importações

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scipy.optimize import fsolve
import casadi as ca
import plotly.graph_objects as go
import optuna
from plotly.subplots import make_subplots
import time

2024-10-16 14:00:29.411234: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 14:00:29.419814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 14:00:29.429417: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 14:00:29.432296: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 14:00:29.439771: I tensorflow/core/platform/cpu_feature_guar

#### Constantes e Variáveis Auxiliares

In [2]:
np.random.seed(42)
print(np.random.seed)

# Constantes
A1 = (2.6)*(10**-3)
Lc = 2
kv = 0.38*100
P1 = 4.5
P_out = 5
C = 479

timestep = 2 # Passos no passado para prever o próximo
epochs = 2000
nData = 600 # Segundos para cada alteração do Alfa
nAlphas = 5 # Número de vezes que o Alfa irá mudar, considere o treino e os testes.
alphas = np.random.uniform(0.35,0.65, nAlphas+1) # Abertura da válvula
perturb = 5e-4
tempo = 60
dt = 0.1 # Tempo amostral

# Variáveis auxiliares
interval = [np.linspace(i * tempo, (i + 1) * tempo, nData) for i in range(nAlphas)]
massFlowrate = []
PlenumPressure = []
alpha_values = []
RNN_train = []
RNN_trainFut = []

<built-in function seed>


### Solução Numérica

##### Cálculo da Solução

In [3]:


# Condições Iniciais
def fun(variables, A1, Lc, kv, P1, P_out, C) :
    (x,y) = variables
    eqn_1 = (A1/Lc)* ((1.5 * P1) - y)*1e6
    eqn_2 = (C**2)/2 * (x - alphas[0] * kv * np.sqrt(y*1000 - P_out*1000))
    return [eqn_1, eqn_2]

result = fsolve(fun, (0, 10), args = (A1, Lc, kv, P1, P_out, C)) 

init_m = result[0] 
init_p = result[1]

# Variáveis CasADi
x = ca.MX.sym('x', 2)
alpha = ca.MX.sym('alpha', 1)

# Solução Numérica
tm1 = time.time()
for i in range(nAlphas):
    alpha_values.append(alphas[i] + np.random.normal(0,perturb,nData))

    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1])*1e6, (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1]*1000 - P_out*1000)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }

    F = ca.integrator('F','idas', ode, interval[0][0], dt)

    for j in range(nData):
        sol = F(x0 = [init_m, init_p], p = alpha_values[i][j])
        xf_values = np.array(sol["xf"])
        aux1, aux2 = xf_values
        massFlowrate.append(aux1)
        PlenumPressure.append(aux2)
        init_m = aux1[-1]
        init_p = aux2[-1]
        RNN_train.append([aux1[0], aux2[0], alpha_values[i][j]])
        RNN_trainFut.append([aux1[0], aux2[0], alpha_values[i][j]])
    
tm2 = time.time()

massFlowrate = np.reshape(massFlowrate, [nAlphas,nData])
PlenumPressure = np.reshape(PlenumPressure, [nAlphas,nData])

print(tm2-tm1)


2.2521049976348877


##### Gráfico do Modelo

In [4]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Vazão vs Tempo", "Pressão vs Tempo", "Alpha vs Tempo"))

for i in range(0, nAlphas):
    # Vazão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrate[i]), mode='lines',
                             name='Vazão', legendgroup='massflow', showlegend=i == 0), row = 1, col = 1)
    # Pressão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressure[i]), mode='lines',
                             name='Pressão', legendgroup='pressure', showlegend=i == 0), row = 1, col = 2)
    # Alphas
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(alpha_values[i]), mode='lines', 
                             name='Alphas', line=dict(dash='dash'), legendgroup='alpha', showlegend=i == 0), row = 1, col = 3)

# Atualiza layout
fig.update_layout(
    xaxis_title='Tempo',
    grid=dict(rows=1, columns=3),
    template='plotly',
    showlegend=False
)

# Mostra a figura
fig.show()


### Rede Neural

##### Dados de Treino

In [5]:
RNN_train = np.array(RNN_train)
RNN_trainFut = np.array(RNN_trainFut)

RNN_train = np.reshape(RNN_train, (nAlphas,3,nData))
RNN_trainFut = np.reshape(RNN_trainFut,(nAlphas,3,nData))

X_train = []
y_train = []

for j in range(nAlphas):
    for i in range(len(RNN_train[0][0])):
        X_train.append(RNN_train[j,:,i])
        if i == (len(RNN_train[0][0]) - 1):
            y_train.append(RNN_trainFut[j, :2, 0])
        else:
            y_train.append(RNN_train[j,:2,i+1])

newData = int((nData*nAlphas)/timestep)

X_train = np.array(X_train)
y_train = np.array(y_train)

X_train = tf.reshape(X_train, [newData,timestep,3])
y_train = tf.reshape(y_train, [newData,timestep,2])

num_samples = X_train.shape[0]
split_index = int(num_samples * 0.8)

X_train_split = X_train[:split_index]
y_train_split = y_train[:split_index]
X_val = X_train[split_index:]
y_val = y_train[split_index:]

I0000 00:00:1729098032.857910  249469 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729098032.881354  249469 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729098032.881500  249469 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729098032.883068  249469 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

##### Rede

In [6]:

class NormLayer(keras.Layer):
    def call(self, inputs):
        batch_min = tf.reduce_min(inputs, axis=[1, 2], keepdims=True)
        batch_max = tf.reduce_max(inputs, axis=[1, 2], keepdims=True)
        return 2 * (inputs - batch_min) / (batch_max - batch_min) - 1, batch_min, batch_max

class DenormLayer(keras.Layer):
    def call(self, inputs, batch_min, batch_max):
        return ((inputs + 1) / 2) * (batch_max - batch_min) + batch_min

class MyModel(tf.keras.Model):
    def __init__(self, num_layers, units):
        super(MyModel, self).__init__()
        self.normalize_layer = NormLayer()
        self.denormalize_layer = DenormLayer()
        self.rnn_layers = []
        self.num_layers = num_layers
        self.units = units
        self.inputs = None

    def build(self, input_shape):
        self.rnn_layers.append(keras.layers.Bidirectional(keras.layers.SimpleRNN(self.units, input_shape = (timestep,3), return_sequences=True, activation='tanh')))
        for _ in range(self.num_layers):
            self.rnn_layers.append(keras.layers.SimpleRNN(self.units, return_sequences=True, activation='tanh'))
        self.dense = keras.layers.Dense(2)

    def call(self, inputs):
        self.inputs = inputs
        normalized_inputs, batch_min, batch_max = self.normalize_layer(inputs)
        rnn_output = normalized_inputs
        for rnn_layer in self.rnn_layers:
            rnn_output = rnn_layer(rnn_output)
        dense_output = self.dense(rnn_output)
        return self.denormalize_layer(dense_output, batch_min, batch_max)

@tf.function
def lossCustom(y_true, y_pred,inputs):
    #m_t = (inputs[:, -2, 0] - 4*y_pred[:, -1, 0] + 3* y_pred[:, -2, 0])/2
    #p_t = (inputs[:, -2, 1] - 4* y_pred[:, -1, 1] + 3*y_pred[:, -2, 1])/2

    #fLoss_mass = tf.reduce_mean(tf.square(m_t - (A1/Lc)*((1.5 * P1) - y_pred[:, -2, 1]) * 1e6))
    #fLoss_pres = tf.reduce_mean(tf.square(p_t - (C**2)/2 * (y_pred[:, -2, 0] - inputs[:, -1, -1]* kv * tf.sqrt((tf.abs(y_pred[:, -2, 1] * 1000 - P_out * 1000))))))
    
    data_loss = tf.reduce_mean(tf.square(y_true[:, :, 0] - y_pred[:, :, 0])) + tf.reduce_mean(tf.square(y_true[:, :, 1] - y_pred[:, :, 1]))
    
    return data_loss #+ 1e-6*(fLoss_mass + fLoss_pres)

def create_model(lr, num_layers, units):
    model = MyModel(num_layers, units)

    def lossCustomWrapper(y_true, y_pred):
        return lossCustom(y_true, y_pred, model.inputs)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr, clipvalue=1.0), loss=lossCustomWrapper)
    return model    

early_stopping = keras.callbacks.EarlyStopping(
    monitor='loss',  
    patience=30000,     
    restore_best_weights=True 
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss',  
    factor=0.5,          
    patience=100,          
    min_lr=1e-8          
)

##### Treinamento

In [7]:
def objective(trial):
    lr = trial.suggest_float('lr', 1e-6, 1e-1, log=True) 
    units = trial.suggest_int('units', 32, 128)
    batch_size = trial.suggest_int('batch_size', 16, 64) 
    num_layers = trial.suggest_int('num_layers', 0, 5)      

    model = create_model(lr, num_layers, units)

    model.fit(X_train_split, y_train_split, epochs=epochs//10, batch_size=batch_size, verbose=0)

    # Avaliação
    loss = model.evaluate(X_val, y_val, verbose=0)
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)  # Ajuste o número de tentativas conforme necessário

# Exibir os melhores hiperparâmetros
print("Melhores hiperparâmetros: ", study.best_params)

[I 2024-10-16 14:00:32,985] A new study created in memory with name: no-name-843ad15c-1fd4-456e-9e85-74d3430f137c
/home/guilhermefreire/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

I0000 00:00:1729098035.124302  249673 service.cc:146] XLA service 0x7b3acc005000 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729098035.124322  249673 service.cc:154]   StreamExecutor device (0): NVIDIA T400 4GB, Compute Capability 7.5
2024-10-16 14:00:35.199834: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-16 14:00:35.450944: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-10-16 14:0

Melhores hiperparâmetros:  {'lr': 7.045773538264988e-05, 'units': 95, 'batch_size': 16, 'num_layers': 4}
Epoch 1/2000
22/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 411194.4688 

2024-10-16 14:04:45.001286: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_4', 24 bytes spill stores, 24 bytes spill loads



24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 118ms/step - loss: 406764.5938 - learning_rate: 7.0458e-05
Epoch 2/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 327662.0938 - learning_rate: 7.0458e-05
Epoch 3/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 312121.5938 - learning_rate: 7.0458e-05
Epoch 4/2000
 1/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 235675.7188

2024-10-16 14:04:47.716458: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_3', 24 bytes spill stores, 24 bytes spill loads



24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 258456.4688 - learning_rate: 7.0458e-05
Epoch 5/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 224516.5781 - learning_rate: 7.0458e-05
Epoch 6/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 228790.8594 - learning_rate: 7.0458e-05
Epoch 7/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 225085.3750 - learning_rate: 7.0458e-05
Epoch 8/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 240015.2500 - learning_rate: 7.0458e-05
Epoch 9/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 243580.8594 - learning_rate: 7.0458e-05
Epoch 10/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 234491.7500 - learning_rate: 7.0458e-05
Epoch 11/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 231383.7500 - learning_rate: 7.0458e-05
Epoch 12/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 229986.7656 - learning_rate: 7.0458e-05
Epoch 13/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 238479.3281 - learning_rate: 7.0458e-05
Epoch 14/

In [ ]:

model = create_model(study.best_params['lr'], study.best_params['num_layers'], study.best_params['units'])

history = model.fit(X_train, y_train, epochs=epochs, verbose=1, batch_size=64, callbacks= [early_stopping, reduce_lr])

##### Dados de teste

In [26]:
massFlowrateTeste = []
PlenumPressureTeste = []
RNN_test = []
x_test = []
alpha_valuesTeste = []
alphasTeste = np.random.uniform(0.35,0.65, nAlphas) # Abertura da válvula
aux1 = []
aux2 = []

tm1 = time.time()
for i in range(0,nAlphas):
    alpha_valuesTeste.append(alphasTeste[i] + np.random.normal(0,perturb,nData))

    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1])*1e6, (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1]*1000 - P_out*1000)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }
    
    for j in range(2):
        F = ca.integrator('F','idas', ode, interval[0][0], dt)
        sol = F(x0 = [init_m, init_p], p = alpha_valuesTeste[i][j])
        xf_values = np.array(sol["xf"])
        aux1, aux2 = xf_values
        massFlowrateTeste.append(aux1)
        PlenumPressureTeste.append(aux2)
        init_m = aux1[-1]
        init_p = aux2[-1]
        RNN_test.append([aux1[0], aux2[0], alpha_valuesTeste[i][j]])

tm2 = time.time()
print(tm2-tm1)

RNN_test = np.array(RNN_test)
RNN_test = np.reshape(RNN_test, (nAlphas,3,2))

x_test.append(RNN_test[:2,:,0])

x_test = np.array(x_test)
x_test = tf.reshape(x_test, [1,timestep,3])

0.018765687942504883


##### Gráfico Rede Neural

In [31]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Loss","Mass Flow Rate vs Time", "Plenum Pressure vs Time"))

# Loss
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)), y=history.history['loss'], mode='lines', name='Mass Loss', line=dict(dash='solid')),
                  row=1, col=1)

mass = []
pressure = []
temp1 = x_test[:,0,:]
temp2 = x_test[:,1,:]

tm1 = time.time()
for i in range(newData):
    inputs = np.array([temp1,temp2])
    prediction = model.predict(inputs)
    temp1 = x_test[:,1,:]
    temp2 = prediction[0,:,:]
    mass.append(prediction[:,0,:])
    pressure.append(prediction[:,0,:])
tm2 = time.time()
print(tm2-tm1)

interval2 = [np.linspace(0, tempo*nAlphas, newData) for i in range(nAlphas)]

for i in range(nAlphas):
    for j in range(0, len(interval2[i]), nData):
        end = j + nData
        if end > len(interval2[i]):
            end = len(interval2[i])
        
        fig.add_trace(go.Scatter(x=interval2[i][j:end],y=np.squeeze(mass[j:end]),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)
        fig.add_trace(go.Scatter(x=interval2[i][j:end],y=np.squeeze(pressure[j:end]),mode='lines',
                                line=dict(dash='solid')),row=1, col=3)
    # Modelo
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrateTeste[i]), mode='lines',name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressureTeste[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=3)

fig.update_layout(
    title='Resultados Rede Neural',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
    showlegend=False
)
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 1) + inhomogeneous part.